In [8]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
from hinge_feature_extraction import * 
from cold_feature_extraction  import * 
from PIL import Image, ImageEnhance
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_val_predict, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

### Reading handwritten images from file and store it with it's lable 
for male y =0 and for female y =1

In [9]:
# feature extraction parameters
opt = {
        'input_folder': r"<IMAGES FOLDER IMAGES ARE STORED>",
        'output_folder': r"<FOLDER WHERE FEATURES ARE STORED",
        'sharpness_factor': 10,
        'bordersize': 3,
        'show_images': False,
        'is_binary': False,
    }

In [10]:
hinge_feature_vector = []
cold_feature_vector =[]
feature_vector =[]
y_data =[]
cold = Cold(opt)
hinge = Hinge(opt)

In [11]:
lables = ["Males" , "Females"]

def feature_extraction(filename):
    data = []
    currentDirectory = os.getcwd()
    # for each class we read it's data 
    for lable in lables:
         directory = filename+"\\"+ lable+"\\"+lable+"\\"
         path =os.path.join(currentDirectory, directory)
         class_lable = lables.index(lable)
         print(str(class_lable) + "-Start ..")
         for imagename in tqdm(os.listdir(path)):
            try:
                img_path = os.path.join(path, imagename)
                hinge_f = hinge.get_hinge_features(img_path)
                cold_f = cold.get_cold_features(img_path)
                hinge_feature_vector.append(hinge_f)
                cold_feature_vector.append(cold_f)
                feature_vector.append([hinge_f , cold_f])
                y_data.append(class_lable)
            except Exception as e :
                print (e)

    return np.array(data)


In [12]:
data = feature_extraction("data_set") 


0-Start ..


  0%|          | 0/232 [00:00<?, ?it/s]c:\Users\LENOVO\Desktop\NN\gender-classification\cold_feature_extraction.py:117: RuntimeWarning: divide by zero encountered in log10
  rhos_log_space = np.log10(rhos)
100%|██████████| 232/232 [31:54<00:00,  8.25s/it]


1-Start ..


100%|██████████| 131/131 [15:22<00:00,  7.04s/it]


### spliting the data with randomness and visualizing the data 

In [13]:
feture_vector = np.concatenate((hinge_feature_vector,cold_feature_vector) , axis=1)
np.shape(hinge_feature_vector) , np.shape(cold_feature_vector) , feture_vector.shape


((363, 780), (363, 420), (363, 1200))

In [14]:
x_train , x_test , y_train , y_test = train_test_split(feture_vector ,y_data , test_size =0.20 , random_state =42)
x_train.shape

(290, 1200)

In [15]:
x_hinge_train = x_train[:,:780]
x_hinge_test = x_test[:,:780]

scaler = StandardScaler()
xtrain = scaler.fit_transform(x_hinge_train)
xtest = scaler.fit_transform(x_hinge_test)

xtrain = np.nan_to_num(xtrain)
xtest = np.nan_to_num(xtest)

clf1 = SVC(kernel='rbf', verbose=True, C=10.0)
clf1.fit(xtrain, y_train)

y_pred_hinge = clf1.predict(xtrain)
y_pred_hinge_test = clf1.predict(xtest)


[LibSVM]

In [16]:
x_cold_train = x_train[:,780:]
x_cold_test = x_test[:,780:]

scaler = StandardScaler()

xtrain = scaler.fit_transform(x_cold_train)
xtest = scaler.fit_transform(x_cold_test)

xtrain = np.nan_to_num(xtrain)
xtest = np.nan_to_num(xtest)

clf2 = SVC(kernel='rbf', verbose=True, C=10.0)
clf2.fit(xtrain, y_train)

y_pred_cold = clf2.predict(xtrain)
y_pred_cold_test = clf2.predict(xtest)


[LibSVM]

In [17]:
y_pred = np.maximum(y_pred_hinge, y_pred_cold)
y_pred_test =np.maximum(y_pred_hinge_test, y_pred_cold_test)

In [18]:
print('Confusion Matrix')
cf = confusion_matrix(y_test, y_pred_test)
cf_sum = cf.sum(axis = 1)[:, np.newaxis]
cf = np.round(cf / cf_sum * 100, 2)
float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})
print(repr(cf))

Confusion Matrix
array([[79.17, 20.83],
       [16.00, 84.00]])


In [19]:
cr = 0.0
for i in range(0, cf.shape[0]):
        cr += cf[i][i]
        
cr /= cf.shape[0]
print(f'classification rate -test = {np.round(cr, 2)}')
    
print(classification_report(y_test, y_pred_test))



classification rate -test = 81.59
              precision    recall  f1-score   support

           0       0.90      0.79      0.84        48
           1       0.68      0.84      0.75        25

    accuracy                           0.81        73
   macro avg       0.79      0.82      0.80        73
weighted avg       0.83      0.81      0.81        73



In [7]:
im = Image.open("data_set\Females\Females\F1.jpg")
# convert to binary image
im.show()
gray = im.convert('1')
gray.show()